In [26]:
import polars as pl
import datetime as dt

In [34]:
(
    pl.read_parquet('./municipio_populacao.parquet')
    .join(
        other=pl.read_excel('../territorio/municipio_area.xlsx'),
        on='municipio_codigo_7',
        how='left'
    )
    .join(
        other=pl.read_excel('../territorio/uf_regiao.xlsx'),
        on='uf_codigo',
        how='left'
    )
    .join(
        other=pl.read_excel('../territorio/uf_codigo.xlsx'),
        on='uf_codigo',
        how='left'
    )
    .join(
        other=pl.read_parquet('../territorio/municipio_completo.parquet').select(pl.col(['municipio_codigo_7','municipio_nome'])),
        on='municipio_codigo_7',
        how='left'
    )
    .with_columns(
        pl.col('populacao')
        .truediv(pl.col('area'))
        .alias('densidade')
    )
    .select(
        'aaaa',
        'regiao',
        'uf_nome',
        'municipio_nome',
        'populacao',
        'area',
        'densidade'
    )
    .rename({
        'aaaa':'ano',
        'uf_nome':'estado',
        'municipio_nome':'municipio',
    })
    # .filter(
    #     pl.col('ano') == dt.datetime(2024,1,1)
    # )
    # .drop_nulls()
    # .sort(['densidade'])
).write_csv('./p-densidade-municipio.csv')

ano,regiao,estado,municipio,populacao,area,densidade
date,str,str,str,i64,f64,f64
2024-01-01,"""Norte""","""Amazonas""","""Barcelos""",18626,122462.433,0.152096
2024-01-01,"""Norte""","""Amazonas""","""Japurá""",9397,55827.202,0.168323
2024-01-01,"""Norte""","""Amazonas""","""Atalaia do Norte""",15892,76507.617,0.207718
2024-01-01,"""Norte""","""Amazonas""","""Santa Isabel do Rio Negro""",14176,62798.845,0.225737
2024-01-01,"""Norte""","""Amazonas""","""Tapauá""",20501,84946.035,0.241341
…,…,…,…,…,…,…
2024-01-01,"""Sudeste""","""São Paulo""","""Carapicuíba""",398462,34.546,11534.244196
2024-01-01,"""Sudeste""","""São Paulo""","""Osasco""",756952,64.954,11653.662592
2024-01-01,"""Sudeste""","""São Paulo""","""Diadema""",404118,30.732,13149.746193
